## FIT3182: Assignment 2 Part A

### Name: Kang Hong Bo (Student Id: 32684673) 

Task 1. MongoDB Data Model

**Question 1:** 
Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, design a suitable data model to support the efficient querying of the two data sets in MongoDB. Justify your data model design. 
The output of this task should be 
- An example of the data model.
- The justification for choosing that data model.


**Answer**

For the data model I choosed embedded data model since this approach minimizes the need for multiple data joins before inserting it into the collection. And for all the requirment for Task 2 it didn't required to merge both data hence we can used embedded document to get all the information within the singular document.

Although embedded document have the possibility that it surpassed the limit given by MongoDB, but since the given data is not that large and it is quite imposible to exceed the limit.

Therefore, the usage of embedded model is suitable in this scenario, which enable efficient document querying without going through all sorts of data joining.

Example of Data Model:

```
 {'GHI_w/m2': 49,
 '_id': ObjectId('663da303cac0c98199ab7957'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2023, 7, 1, 0, 0),
 'hotspots': [{'confidence': 53,
               'datetime': datetime.datetime(2023, 7, 1, 13, 11, 41),
               'latitude': -37.062,
               'longitude': 141.373,
               'surface_temperature_celcius': 29},
              {'confidence': 53,
               'datetime': datetime.datetime(2023, 7, 1, 13, 11, 41),
               'latitude': -37.062,
               'longitude': 141.373,
               'surface_temperature_celcius': 29},
              {'confidence': 61,
               'datetime': datetime.datetime(2023, 7, 1, 3, 46, 8),
               'latitude': -36.779,
               'longitude': 146.108,
               'surface_temperature_celcius': 32},
              {'confidence': 61,
               'datetime': datetime.datetime(2023, 7, 1, 3, 46, 8),
               'latitude': -36.779,
               'longitude': 146.108,
               'surface_temperature_celcius': 32}],
 'max_wind_speed': 14.0,
 'precipitation': 0.0,
 'precipitationFlag': 'I',
 'relative_humidity': 33.1,
 'station': 948701,
 'windspeed_knots': 5.8}
 ```







Upgrade pymongo for latest version

In [60]:
!pip install --upgrade pymongo


Task 2: Q1. Querying MongoDB using PyMongo

In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient('192.168.22.110',27017) ## PLEASE CHANGE THIS LINE TO YOUR HOSTIP
import pandas as pd
import csv
from datetime import datetime
from pprint import pprint
import time

db= client.fit3182_assignment_db
collection = db.partA
collection.drop() # Remove all data that have been added into collection

# Read data
hotspot = pd.read_csv('hotspot_historic.csv')
climate = pd.read_csv('climate_historic.csv')

#Make all date and datetime data to datetime format
climate['date'] = climate['date'].apply(lambda x:datetime.strptime(x,'%d/%m/%Y'))
hotspot['date'] = hotspot['date'].apply(lambda x:datetime.strptime(x,'%d/%m/%Y'))

#
hotspot['datetime'] = hotspot['datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

climate['precipitation_flag'] = climate['precipitation'].str[-1]
climate['precipitation'] = climate['precipitation'].str[0:-1]

In [2]:
for iter,climateRow in climate.iterrows():
    document={}
    document['station'] = int(climateRow['station'])
    document['date'] = climateRow['date'] 
    document['air_temperature_celcius'] = int(climateRow['air_temperature_celcius'])
    document['relative_humidity'] = float(climateRow['relative_humidity'])
    document['windspeed_knots'] = float(climateRow['windspeed_knots'])
    document['max_wind_speed'] = float(climateRow['max_wind_speed'])
    document['precipitationFlag'] = climateRow['precipitation_flag'].strip()
    document['precipitation']=float(climateRow['precipitation'].strip())
    document['GHI_w/m2'] = int(climateRow['GHI_w/m2'])
    document['hotspots']=[]
    #locate all hotspots with same date with the climate
    hotspotList = hotspot.loc[hotspot['date'] == climateRow['date']]
    
    
    # If we have one or more hotspot that have the same date as current climate data
    if len(hotspotList) >= 1:
#         document['hotspots'] = [] # Hold a list of hotspot documents where all the hotspot's date matches current climate date
        
        # Put each matching hotspot data into a document
        for index2,hotspotRow in hotspotList.iterrows():
            hotspotDoc = {}
            hotspotDoc['latitude'] = float(hotspotRow['latitude'])
            hotspotDoc['longitude'] = float(hotspotRow['longitude'])
            hotspotDoc['datetime'] = hotspotRow['datetime']
            hotspotDoc['confidence'] = int(hotspotRow['confidence'])
            hotspotDoc['surface_temperature_celcius'] = int(hotspotRow['surface_temperature_celcius'])
            document['hotspots'].append(hotspotDoc)
    
    collection.insert_one(document)

    

Q2A: Find climate data on 12th December 2023. 

In [3]:
toFind = datetime.strptime('12/12/2023','%d/%m/%Y')
sol= collection.find_one({"date":toFind})
pprint(sol)

{'GHI_w/m2': 156,
 '_id': ObjectId('664b41af39b4d66d1f542dea'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2023, 12, 12, 0, 0),
 'hotspots': [{'confidence': 53,
               'datetime': datetime.datetime(2023, 12, 12, 0, 45, 38),
               'latitude': -37.903,
               'longitude': 145.25,
               'surface_temperature_celcius': 44}],
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitationFlag': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


Q2B. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [4]:
sol = collection.aggregate([
    {"$unwind":"$hotspots"},
    {"$match":{"$and":[{"hotspots.surface_temperature_celcius":{"$gte":65}},
                       {"hotspots.surface_temperature_celcius":{"$lte":100}}]}},
    {"$project":{"hotspots.latitude":1,
                 "hotspots.longitude":1,
                 "hotspots.surface_temperature_celcius":1,
                 "hotspots.confidence":1,"_id":0}}
])
for documents in sol:
    pprint(documents)

{'hotspots': {'confidence': 94,
              'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celcius': 73}}
{'hotspots': {'confidence': 97,
              'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celcius': 80}}
{'hotspots': {'confidence': 84,
              'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celcius': 71}}
{'hotspots': {'confidence': 100,
              'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celcius': 99}}
{'hotspots': {'confidence': 80,
              'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celcius': 68}}
{'hotspots': {'confidence': 85,
              'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celcius': 98}}
{'hotspots': {'confidence': 90,
              'latitude': -34.357,
   

Q2C. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2023. 

In [5]:
sol = collection.aggregate([
    {"$match":{"$or":[{"date":datetime.strptime('15/12/2023','%d/%m/%Y')},
                      {"date":datetime.strptime('16/12/2023','%d/%m/%Y')}]}},
    {"$project":{"date":1,"hotspots.surface_temperature_celcius":1,
                "air_temperature_celcius":1,
                 "relative_humidity":1,
                 "max_wind_speed":1,
                 "_id":0}}
])

for doc in sol:
    pprint(doc)

{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2023, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_temperature_celcius': 60},
              {'surface_temperature_celcius': 73},
              {'surfa

Q2D.Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [6]:
sol = collection.aggregate([
    {"$match":{"$and":[{"hotspots.confidence":{"$gte":80}},
                       {"hotspots.confidence":{"$lte":100}}]}},
    {"$project":{"hotspots.datetime":1,
                 "air_temperature_celcius":1,
                 "hotspots.surface_temperature_celcius":1,
                 "hotspots.confidence":1,"_id":0}}
])
for documents in sol:
    pprint(documents)

{'air_temperature_celcius': 20,
 'hotspots': [{'confidence': 87,
               'datetime': datetime.datetime(2023, 3, 6, 5, 6, 30),
               'surface_temperature_celcius': 62},
              {'confidence': 85,
               'datetime': datetime.datetime(2023, 3, 6, 5, 6, 20),
               'surface_temperature_celcius': 59}]}
{'air_temperature_celcius': 19,
 'hotspots': [{'confidence': 88,
               'datetime': datetime.datetime(2023, 3, 7, 4, 16, 10),
               'surface_temperature_celcius': 64}]}
{'air_temperature_celcius': 23,
 'hotspots': [{'confidence': 86,
               'datetime': datetime.datetime(2023, 3, 9, 13, 23, 40),
               'surface_temperature_celcius': 41},
              {'confidence': 54,
               'datetime': datetime.datetime(2023, 3, 9, 3, 57),
               'surface_temperature_celcius': 44},
              {'confidence': 78,
               'datetime': datetime.datetime(2023, 3, 9, 3, 56, 50),
               'surface_temperature_celc

 'hotspots': [{'confidence': 78,
               'datetime': datetime.datetime(2023, 5, 15, 4, 36, 40),
               'surface_temperature_celcius': 51},
              {'confidence': 64,
               'datetime': datetime.datetime(2023, 5, 15, 4, 35, 20),
               'surface_temperature_celcius': 42},
              {'confidence': 59,
               'datetime': datetime.datetime(2023, 5, 15, 4, 34, 50),
               'surface_temperature_celcius': 40},
              {'confidence': 93,
               'datetime': datetime.datetime(2023, 5, 15, 4, 34, 30),
               'surface_temperature_celcius': 73},
              {'confidence': 96,
               'datetime': datetime.datetime(2023, 5, 15, 4, 34, 30),
               'surface_temperature_celcius': 78},
              {'confidence': 76,
               'datetime': datetime.datetime(2023, 5, 15, 4, 33, 50),
               'surface_temperature_celcius': 50},
              {'confidence': 69,
               'datetime': datetime.datetim

              {'confidence': 100,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 10),
               'surface_temperature_celcius': 47},
              {'confidence': 59,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 9),
               'surface_temperature_celcius': 35},
              {'confidence': 82,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 9),
               'surface_temperature_celcius': 38},
              {'confidence': 100,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 9),
               'surface_temperature_celcius': 48},
              {'confidence': 100,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 9),
               'surface_temperature_celcius': 48},
              {'confidence': 84,
               'datetime': datetime.datetime(2023, 9, 24, 13, 30, 7),
               'surface_temperature_celcius': 50},
              {'confidence': 100,
               'datetime': datetime.da

Q2E.Find the top 10 records with the highest surface temperature (°C).

In [7]:
sol = collection.aggregate([
    {"$unwind":"$hotspots"},
    {"$sort":{"hotspots.surface_temperature_celcius":-1}},
    {"$limit":10}
])
for doc in sol:
    pprint(doc)

{'GHI_w/m2': 122,
 '_id': ObjectId('663da303cac0c98199ab790d'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2023, 4, 18, 0, 0),
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2023, 4, 18, 4, 52),
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation': 0.0,
 'precipitationFlag': 'I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('663da303cac0c98199ab78ff'),
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2023, 4, 4, 0, 0),
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2023, 4, 4, 4, 32, 50),
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitationFlag': 'I',
 'relative_humidity': 47.5,
 'station': 948701,


Q2F. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

In [8]:
sol=collection.aggregate([
    {"$project":{
        "_id":0,
        "date":1,
        "numberOfFires":{
            "$size":"$hotspots"
            }
        }      
    }

])
for document in sol:
    document['date'] = document['date'].strftime('%d/%m/%Y')
    pprint(document)

{'date': '31/12/2022', 'numberOfFires': 0}
{'date': '02/01/2023', 'numberOfFires': 0}
{'date': '03/01/2023', 'numberOfFires': 0}
{'date': '04/01/2023', 'numberOfFires': 0}
{'date': '05/01/2023', 'numberOfFires': 0}
{'date': '06/01/2023', 'numberOfFires': 0}
{'date': '07/01/2023', 'numberOfFires': 0}
{'date': '08/01/2023', 'numberOfFires': 0}
{'date': '09/01/2023', 'numberOfFires': 0}
{'date': '10/01/2023', 'numberOfFires': 0}
{'date': '11/01/2023', 'numberOfFires': 0}
{'date': '12/01/2023', 'numberOfFires': 0}
{'date': '13/01/2023', 'numberOfFires': 0}
{'date': '14/01/2023', 'numberOfFires': 0}
{'date': '15/01/2023', 'numberOfFires': 0}
{'date': '16/01/2023', 'numberOfFires': 0}
{'date': '17/01/2023', 'numberOfFires': 0}
{'date': '18/01/2023', 'numberOfFires': 0}
{'date': '19/01/2023', 'numberOfFires': 0}
{'date': '20/01/2023', 'numberOfFires': 0}
{'date': '21/01/2023', 'numberOfFires': 0}
{'date': '22/01/2023', 'numberOfFires': 0}
{'date': '23/01/2023', 'numberOfFires': 0}
{'date': '2

Q2G. Find the records of fires where the confidence is below 70.

In [9]:
sol = collection.aggregate([
    {"$unwind":"$hotspots"},
    {"$match": {"hotspots.confidence":{"$lte":70}}},
    {"$project":{"hotspots":1,"_id":0}}
])
for document in sol:
    document['hotspots']['datetime'] = document['hotspots']['datetime'].strftime('%H:%M:%S %d/%m/%Y')
    pprint(document)

{'hotspots': {'confidence': 68,
              'datetime': '04:51:00 08/03/2023',
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55}}
{'hotspots': {'confidence': 54,
              'datetime': '03:57:00 09/03/2023',
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 55,
              'datetime': '04:43:00 10/03/2023',
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 54,
              'datetime': '04:42:30 10/03/2023',
              'latitude': -37.2197,
              'longitude': 147.9621,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 56,
              'datetime': '23:58:50 13/03/2023',
              'latitude': -37.0286,
              'longitude': 148.155,
              'surface_temperat

Q2H. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [10]:
sol = collection.aggregate([{"$unwind":"$hotspots"},
    {"$group":{"_id":"$date","avg_surface_temperature":{"$avg":"$hotspots.surface_temperature_celcius"}}},
    {"$project":{"date":"$_id",
                 "_id":0,
                 "avg_surface_temperature":1}}
])
for document in sol:
    document['date'] = document['date'].strftime('%d/%m/%Y')
    pprint(document)


{'avg_surface_temperature': 49.0, 'date': '09/06/2023'}
{'avg_surface_temperature': 53.57142857142857, 'date': '24/09/2023'}
{'avg_surface_temperature': 32.0, 'date': '24/12/2023'}
{'avg_surface_temperature': 50.0, 'date': '20/10/2023'}
{'avg_surface_temperature': 54.5, 'date': '22/04/2023'}
{'avg_surface_temperature': 61.5, 'date': '22/11/2023'}
{'avg_surface_temperature': 43.0, 'date': '29/09/2023'}
{'avg_surface_temperature': 51.5, 'date': '08/03/2023'}
{'avg_surface_temperature': 48.72222222222222, 'date': '16/04/2023'}
{'avg_surface_temperature': 51.857142857142854, 'date': '07/06/2023'}
{'avg_surface_temperature': 53.25, 'date': '14/06/2023'}
{'avg_surface_temperature': 50.4, 'date': '27/10/2023'}
{'avg_surface_temperature': 50.333333333333336, 'date': '07/05/2023'}
{'avg_surface_temperature': 45.5, 'date': '08/11/2023'}
{'avg_surface_temperature': 61.3, 'date': '09/11/2023'}
{'avg_surface_temperature': 57.8, 'date': '16/12/2023'}
{'avg_surface_temperature': 54.16, 'date': '19/04

Q2I. Find the top 10 records with the lowest GHI.

In [11]:
sol= collection.aggregate([
    {"$sort":{"GHI_w/m2":1}},
    {"$limit":10}
])

for doc in sol:
    pprint(doc)

{'GHI_w/m2': 47,
 '_id': ObjectId('663da303cac0c98199ab7977'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2023, 8, 2, 0, 0),
 'hotspots': [{'confidence': 94,
               'datetime': datetime.datetime(2023, 8, 2, 3, 45, 40),
               'latitude': -37.4796,
               'longitude': 141.9403,
               'surface_temperature_celcius': 87},
              {'confidence': 54,
               'datetime': datetime.datetime(2023, 8, 2, 3, 45),
               'latitude': -37.491,
               'longitude': 141.936,
               'surface_temperature_celcius': 40}],
 'max_wind_speed': 5.1,
 'precipitation': 0.0,
 'precipitationFlag': 'I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('663da303cac0c98199ab7956'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2023, 6, 30, 0, 0),
 'hotspots': [{'confidence': 78,
               'datetime': datetime.datetime(2023, 6, 30, 4, 41, 25),
               'la

Q2J. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.

In [25]:
sol = collection.find(
    {"precipitationFlag":"G","precipitation":{"$gte":0.20,"$lte":0.35}}
    
)
for doc in sol:
    pprint(doc)

{'GHI_w/m2': 157,
 '_id': ObjectId('663da303cac0c98199ab78ae'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2023, 1, 13, 0, 0),
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': 0.31,
 'precipitationFlag': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('663da303cac0c98199ab78f9'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2023, 3, 29, 0, 0),
 'hotspots': [{'confidence': 69,
               'datetime': datetime.datetime(2023, 3, 29, 0, 48, 40),
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitationFlag': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('663da303cac0c98199ab790f'),
 'air_temperature_celcius': 20,
 'date': datetime.datetime(2023, 4, 20, 0, 0),
 'hotspots': [{'confiden

**Task 3 - Index Design**

Based on the indexing strategies I tend to use The ESR (Equal,Sort,Range) Rule to add in indexes based on all the quries above.

Before adding new indexes remove all of the indexes for testing purpose below.

In [49]:
collection.drop_indexes()

Based on Q2A the explain methods gives us a straight view on how it get the data in the winning plan and we can observe that it went through 366 document in the data called 'totalDocsExamines'

In [11]:
toFind = datetime.strptime('12/12/2023','%d/%m/%Y')
collection.find({"date":toFind}).explain()

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'fit3182_assignment_db.partA',
  'indexFilterSet': False,
  'parsedQuery': {'date': {'$eq': datetime.datetime(2023, 12, 12, 0, 0)}},
  'queryHash': '6C31DBA7',
  'planCacheKey': '6C31DBA7',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'date': {'$eq': datetime.datetime(2023, 12, 12, 0, 0)}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 1,
  'executionTimeMillis': 0,
  'totalKeysExamined': 0,
  'totalDocsExamined': 366,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'date': {'$eq': datetime.datetime(2023, 12, 12, 0, 0)}},
   'nReturned': 1,
   'executionTimeMillisEstimate': 0,
   'works': 368,
   'advanced': 1,
   'needTime': 366,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',


After adding the date index, we can see that the examined docs decrease to 1 instead.

In [65]:
collection.create_index({"date":1})
toFind = datetime.strptime('12/12/2023','%d/%m/%Y')
collection.find({"date":toFind}).explain()
# pprint(sol)

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'fit3182_assignment_db.partA',
  'indexFilterSet': False,
  'parsedQuery': {'date': {'$eq': datetime.datetime(2023, 12, 12, 0, 0)}},
  'queryHash': '6C31DBA7',
  'planCacheKey': '39CDD611',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'date': 1},
    'indexName': 'date_1',
    'isMultiKey': False,
    'multiKeyPaths': {'date': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'date': ['[new Date(1702339200000), new Date(1702339200000)]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 1,
  'executionTimeMillis': 2,
  'totalKeysExamined': 1,
  'totalDocsExamined': 1,
  'executionStages': {'stage': 'FETCH',
   'nReturned': 1,
   'execu

For Q2B, the query include range query with the uses of hotspots.surface_temperature_celcius so I added a simple index. And since I used aggregate and it doens't have the method explain so to indicate the scenario of matching the data set i used find to get the explain on how many documents it examined

In [77]:
collection.drop_indexes()
pprint(collection.find({"hotspots.surface_temperature_celcius":{"$gte":65,"$lte":100}}).explain()['executionStats'])


{'allPlansExecution': [],
 'executionStages': {'advanced': 66,
                     'direction': 'forward',
                     'docsExamined': 366,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'$and': [{'hotspots.surface_temperature_celcius': {'$lte': 100}},
                                         {'hotspots.surface_temperature_celcius': {'$gte': 65}}]},
                     'isEOF': 1,
                     'nReturned': 66,
                     'needTime': 301,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 368},
 'executionSuccess': True,
 'executionTimeMillis': 0,
 'nReturned': 66,
 'totalDocsExamined': 366,
 'totalKeysExamined': 0}


In [79]:
collection.create_index({"hotspots.surface_temperature_celcius":1})


pprint(collection.find({"hotspots.surface_temperature_celcius":{"$gte":65,"$lte":100}}).explain()["executionStats"])


{'allPlansExecution': [{'executionStages': {'advanced': 66,
                                            'alreadyHasObj': 0,
                                            'docsExamined': 66,
                                            'executionTimeMillisEstimate': 0,
                                            'filter': {'hotspots.surface_temperature_celcius': {'$lte': 100}},
                                            'inputStage': {'advanced': 66,
                                                           'direction': 'forward',
                                                           'dupsDropped': 361,
                                                           'dupsTested': 427,
                                                           'executionTimeMillisEstimate': 0,
                                                           'indexBounds': {'hotspots.surface_temperature_celcius': ['[65, '
                                                                                           

From the two results of execitionStages we can see that for the first one without indexing it go through 366 document but the one with indexing went through 66. Which reduced the searching range of it.

For Q2C, it queries for certain date data so it uses the same index as Q2A

For Q2D, it uses Confidence to query for a specific range so I include a new simple index for confidence. Using the same approach above to show how it will impact.

In [80]:
collection.drop_indexes()
pprint(collection.find({"hotspots.confidence":{"$gte":80,"$lte":100}}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 103,
                     'direction': 'forward',
                     'docsExamined': 366,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'$and': [{'hotspots.confidence': {'$lte': 100}},
                                         {'hotspots.confidence': {'$gte': 80}}]},
                     'isEOF': 1,
                     'nReturned': 103,
                     'needTime': 264,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 368},
 'executionSuccess': True,
 'executionTimeMillis': 1,
 'nReturned': 103,
 'totalDocsExamined': 366,
 'totalKeysExamined': 0}


In [81]:
collection.create_index({"hotspots.confidence":1})
pprint(collection.find({"hotspots.confidence":{"$gte":80,"$lte":100}}).explain()['executionStats'])

{'allPlansExecution': [{'executionStages': {'advanced': 101,
                                            'alreadyHasObj': 0,
                                            'docsExamined': 101,
                                            'executionTimeMillisEstimate': 0,
                                            'filter': {'hotspots.confidence': {'$lte': 100}},
                                            'inputStage': {'advanced': 101,
                                                           'direction': 'forward',
                                                           'dupsDropped': 395,
                                                           'dupsTested': 496,
                                                           'executionTimeMillisEstimate': 0,
                                                           'indexBounds': {'hotspots.confidence': ['[80, '
                                                                                                   'inf.0]']},
           

From the results of execution stages above, <br>
non-index: 'totalDocsExamined': 366,<br>
indexing: 'totalDocsExamined': 103<br>


For Q2E we are getting the top 10 largest surface temperature which means we need to sort the value and hence we could add a new index with the same order as the query.

In [11]:
collection.drop_indexes()
explain_result = collection.find(
    {"hotspots.surface_temperature_celcius": {}}
).sort("hotspots.surface_temperature_celcius", pymongo.DESCENDING).limit(10).explain()

pprint(explain_result['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 0,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 0,
                                    'direction': 'forward',
                                    'docsExamined': 366,
                                    'executionTimeMillisEstimate': 0,
                                    'filter': {'hotspots.surface_temperature_celcius': {'$eq': {}}},
                                    'isEOF': 1,
                                    'nReturned': 0,
                                    'needTime': 367,
                                    'needYield': 0,
                                    'restoreState': 0,
                                    'saveState': 0,
                                    'stage': 'COLLSCAN',
                                    'works': 368},
                     'isEOF': 1,
                     'limitAmount': 10,
                     'memLimit': 104857600,
       

In [15]:
collection.create_index({"hotspots.surface_temperature_celcius":-1})
explain_result = collection.find(
    {"hotspots.surface_temperature_celcius": {}}
).sort("hotspots.surface_temperature_celcius", pymongo.DESCENDING).limit(10).explain()

pprint(explain_result['executionStats'])

{'allPlansExecution': [{'executionStages': {'advanced': 0,
                                            'executionTimeMillisEstimate': 0,
                                            'inputStage': {'advanced': 0,
                                                           'alreadyHasObj': 0,
                                                           'docsExamined': 0,
                                                           'executionTimeMillisEstimate': 0,
                                                           'inputStage': {'advanced': 0,
                                                                          'direction': 'forward',
                                                                          'dupsDropped': 0,
                                                                          'dupsTested': 0,
                                                                          'executionTimeMillisEstimate': 0,
                                                             

For the results above,<br>
Without Indexing: 'totalDocsExamined': 366<br>
With Indexing: 'totalDocsExamined': 0 <br>
This relative indexing provide the direct data for query so 0 Docs were examined

For Q2F required to quries of the fires amount. And its just retriving the size of the hotspots data so its doesn't include Equal, sort or range query. So it doesn't required any index for it.

For Q2G Its query for range of confidence and since above Q2D added the index for confidence we don't need to add any others index

For Q2H its doesn't use query which related to equal, sort, nor range so I didn't add index for this task.

For Q2I its uses top 10 records with the lowest GHI. Which means it needs sorting for querying and since its sort we can add indexing for it. 

In [16]:
collection.drop_indexes()
explain_result = collection.find(
    {"GHI_w/m2": {}}
).sort("GHI_w/m2", pymongo.ASCENDING).limit(10).explain()

pprint(explain_result['executionStats'])



{'allPlansExecution': [],
 'executionStages': {'advanced': 0,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 0,
                                    'direction': 'forward',
                                    'docsExamined': 366,
                                    'executionTimeMillisEstimate': 0,
                                    'filter': {'GHI_w/m2': {'$eq': {}}},
                                    'isEOF': 1,
                                    'nReturned': 0,
                                    'needTime': 367,
                                    'needYield': 0,
                                    'restoreState': 0,
                                    'saveState': 0,
                                    'stage': 'COLLSCAN',
                                    'works': 368},
                     'isEOF': 1,
                     'limitAmount': 10,
                     'memLimit': 104857600,
                     'nReturned': 0

In [17]:
collection.create_index({"GHI_w/m2":1})
explain_result = collection.find(
    {"GHI_w/m2": {}}
).sort("GHI_w/m2", pymongo.ASCENDING).limit(10).explain()

pprint(explain_result['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 0,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 0,
                                    'alreadyHasObj': 0,
                                    'docsExamined': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'inputStage': {'advanced': 0,
                                                   'direction': 'forward',
                                                   'dupsDropped': 0,
                                                   'dupsTested': 0,
                                                   'executionTimeMillisEstimate': 0,
                                                   'indexBounds': {'GHI_w/m2': ['[{}, '
                                                                                '{}]']},
                                                   'indexName': 'GHI_w/m2_1',
                                 

The results is the same as Q2E <br>
With Indexing: 'totalDocsExamined': 366 <br>
Without Indexing: 'totalDocsExamined': 0

For Q2J it uses precipitaionFlag and precipitation to find so we can add a compound index for it.

In [19]:
collection.drop_indexes()
sol = collection.find(
    {"precipitationFlag":"G","precipitation":{"$gte":0.20,"$lte":0.35}}
).explain()
pprint(sol["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 19,
                     'direction': 'forward',
                     'docsExamined': 366,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'$and': [{'precipitationFlag': {'$eq': 'G'}},
                                         {'precipitation': {'$lte': 0.35}},
                                         {'precipitation': {'$gte': 0.2}}]},
                     'isEOF': 1,
                     'nReturned': 19,
                     'needTime': 348,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 368},
 'executionSuccess': True,
 'executionTimeMillis': 0,
 'nReturned': 19,
 'totalDocsExamined': 366,
 'totalKeysExamined': 0}


In [20]:
collection.create_index({"precipitationFlag":1,"precipitation":1})
sol = collection.find(
    {"precipitationFlag":"G","precipitation":{"$gte":0.20,"$lte":0.35}}
).explain()
pprint(sol["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 19,
                     'alreadyHasObj': 0,
                     'docsExamined': 19,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 19,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'precipitation': ['[0.2, '
                                                                      '0.35]'],
                                                    'precipitationFlag': ['["G", '
                                                                          '"G"]']},
                                    'indexName': 'precipitationFlag_1_precipitation_1',
                                    'indexVersion': 2,
                                    'isEOF

From the results above,<br>
Without index: 'totalDocsExamined': 366 <br>
With Index: 'totalDocsExamined': 19